In [1]:
import pyspark
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

!sed -i 's/hadoop.root.logger=INFO,console/hadoop.root.logger=WARN,console/' /usr/hadoop-3.3.2/etc/hadoop/log4j.properties
warnings.filterwarnings('ignore')

conf = pyspark.SparkConf().setAll(
    [('spark.master', 'local[2]'),
     ('spark.app.name', 'Cleaning')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

2023-05-27 18:19:04,179 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-05-27 18:19:05,581 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. Load Dataset

The dataset was procured as a csv from the [City of Chicago](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present-Dashboard/5cd6-ry5g). To keep even distribution of data across seasons, 2023 data will be removed immediately (incomplete year).

In [2]:
# csv was renamed to `crime.csv`
path = 'home/work/Project/mas-dse-230/crime.csv'

df = spark.read.csv(f'file:///{path}', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
df = df.filter("Year < 2023")

print(f'Rows: {df.count()}, Columns: {len(df.schema)}')

Rows: 7711262, Columns: 22


### 2. Null Handling

It's possible that the distribution of null is time-dependent due to changing data practices. As 7.7M records is already plenty of data, imbalanced years can likely be dropped with minimal impact so long as it comes from an extreme of the dataset.

In [7]:
denulled = df.dropna().groupBy('Year').count()
denulled.createOrReplaceTempView('denulled')

query = '''
    WITH cte AS (
        SELECT Year, COUNT(*) AS total_records
        FROM df
        GROUP BY Year
    )
    SELECT cte.Year, cte.total_records, denulled.count AS records_without_nulls
        , ROUND(1 - denulled.count/cte.total_records, 4) AS proportion_removed
    FROM cte
    INNER JOIN denulled
        ON denulled.Year = cte.Year
    ORDER BY Year DESC
'''
spark.sql(query).show(n=100)

+----+-------------+---------------------+------------------+
|Year|total_records|records_without_nulls|proportion_removed|
+----+-------------+---------------------+------------------+
|2022|       238017|               232561|            0.0229|
|2021|       208543|               202246|            0.0302|
|2020|       212080|               207439|            0.0219|
|2019|       261242|               258161|            0.0118|
|2018|       268769|               262834|            0.0221|
|2017|       269064|               264156|            0.0182|
|2016|       269785|               266482|            0.0122|
|2015|       264755|               257788|            0.0263|
|2014|       275731|               273610|            0.0077|
|2013|       307466|               306325|            0.0037|
|2012|       336261|               335099|            0.0035|
|2011|       351960|               350905|             0.003|
|2010|       370496|               369805|            0.0019|
|2009|  

Nearly all of 2001 had records with nulls. 2002 had a significant proportion of null-containing records as well (29%).

These years will be removed as will all null records since they account for a relatively small proportion of the total dataset. This null removall may impact the accuracy of downstream analyses.

In [9]:
df = df.filter("Year > 2002").dropna()
df.createOrReplaceTempView('df')

df.count()

6663080

### 3. Category Consolidation

The features `Primary Type` and `Description` (to be further explored in the next notebook) contain categorical values that describe a crime. Many of these labels appear to be overlapping or text variations of each other, for example, `NON-CRIMINAL` and `NON - CRIMINAL`.

The next cells will consolidate these categories.

In [13]:
ptype_consolidation = {
    'CRIM SEXUAL ASSAULT': 'CRIMINAL SEXUAL ASSAULT',
    'NON-CRIMINAL (SUBJECT SPECIFIED)': 'NON-CRIMINAL',
    'NON - CRIMINAL': 'NON-CRIMINAL',
    'OTHER NARCOTIC VIOLATION': 'NARCOTICS',
    'PUBLIC INDECENCY': 'PUBLIC INDECENCY/OBSCENITY',
    'OBSCENITY': 'PUBLIC INDECENCY/OBSCENITY'
}
desc_consolidation = {
    'AGGRAVATED':'AGG',
    'ATTEMPT':'ATT',
    'CRIMINAL':'CRIM',
    'POSSESSION':'POSS',
    'POS ':'POSS',
    'POSESS:':'POSS ',
    'POSESS ':'POSS ',    
    'REGISTRATION':'REG',
    'PRO. ':'PROFESSIONAL ',
    'PO ': 'POLICE OFFICER ',
    'P.O.':'POLICE OFFICER',
    'RITUAL': 'RIT',
    'MUTILATION':'MUT',
    'INSTRUMENT':'INSTR',
    'MANU/DEL:':'MANUFACTURE DELIVER',
    'MANU/DELIVER:':'MANUFACTURE DELIVER',
    'MANUFACTURE / DELIVER':'MANUFACTURE DELIVER'
}

for k, v in ptype_consolidation.items():
    df = df.replace(k, v, 'Primary Type')
    
replacement_expr = col('Description')
for k, v in desc_consolidation.items():
    replacement_expr = regexp_replace(replacement_expr, k, v)
df = df.withColumn('Description', replacement_expr)
df = df.withColumn('Description', regexp_replace(df['Description'], '[^A-Za-z0-9]', ''))

Cleaning is now done, specify the output path to generate the new csv which will be the input for the next notebooks.

In [15]:
write_path = 'home/work/Project/mas-dse-230/cleaned_crime.csv'
df.coalesce(1).write.mode('overwrite').csv(f'file:///{write_path}')

In [16]:
spark.stop()